In [ ]:
#!pip install opencv-python imgaug numpy

In [ ]:
#!pip install moviepy

In [ ]:
import cv2
import numpy as np
import moviepy.editor as mp
import os

def augment_video(input_file, output_dir, num_augmentations=5):
    try:
        video = mp.VideoFileClip(input_file)
        fps = video.fps
    except Exception as e:
        print(f"Error loading video with moviepy {input_file}: {e}")
        return
    
    # If None, try OpenCV
    if fps is None:
        print(f"Warning: FPS could not be determined by moviepy for {input_file}. Trying with OpenCV.")
        cap = cv2.VideoCapture(input_file)
        fps = cap.get(cv2.CAP_PROP_FPS)
        cap.release()
        
    # If None, set a default FPS
    if fps is None or fps == 0:
        print(f"Warning: FPS could not be determined for {input_file}. Using default FPS of 24.")
        fps = 24  # Default to 24 fps if undefined
    
    # Generate augmented videos
    for i in range(num_augmentations):
        params = generate_augmentation_params()
        
        clip = video.fl_image(lambda frame: augment_frame(frame, params))
        
        base_name = os.path.basename(input_file)
        name, ext = os.path.splitext(base_name)
        output_file = os.path.join(output_dir, f"{name}_augmented_{i}{ext}")
        
        try:
            clip.write_videofile(output_file, codec='libx264', fps=fps)
        except Exception as e:
            print(f"Error writing video {output_file}: {e}")

def generate_augmentation_params():
    # Generate random parameters for augmentation
    brightness_factor = np.random.uniform(0.5, 1.5)  
    rotation_angle = np.random.uniform(-30, 30)  
    scale_factor = np.random.uniform(0.8, 1.2)  
    return brightness_factor, rotation_angle, scale_factor

def augment_frame(frame, params):
    brightness_factor, rotation_angle, scale_factor = params
    
    frame = cv2.convertScaleAbs(frame, alpha=brightness_factor, beta=0)
    
    frame = frame.astype(np.float32)
    
    (h, w) = frame.shape[:2]
    center = (w / 2, h / 2)
    matrix = cv2.getRotationMatrix2D(center, rotation_angle, scale_factor)
    frame = cv2.warpAffine(frame, matrix, (w, h), flags=cv2.INTER_LINEAR)
    
    frame = np.clip(frame, 0, 255).astype(np.uint8)
    
    return frame

def process_directory(input_dir, num_augmentations=5):
    # Traverse the directory tree
    for root, dirs, files in os.walk(input_dir):
        for file_name in files:
            if file_name.lower().endswith(('.mp4', '.avi', '.mov', '.mkv')):
                input_file = os.path.join(root, file_name)
                
                # Saving augmented videos in the same directory
                output_dir = root
                
                print(f"Processing {input_file}...")
                augment_video(input_file, output_dir, num_augmentations)
                print(f"Augmented videos saved in {output_dir}")

input_directory = r'C:\Users\USER\Desktop\SIH\Code\ds1'

process_directory(input_directory)
